# Quick and dirty conversion of notebooks to js scripts

Borrowing from [nbdev](https://github.com/fastai/nbdev/), this notebook will create `nbdev_js.py`. 

Use `nbdev_js.py` to convert notebooks to js scripts.

You can convert all notebooks, in the current working directory, programatically;
```
from nbdev_js import notebook2script
notebook2script()
```
or from the command line;
```
python nbdev_js.py
```
By default, `nbdev_js.py` will;
- read all notebook files that match the glob pattern `'./[!_]*.ipynb'`
    - all notebooks, in the current working directory, that don't start with `_`
    - this can be overriden via the `fname` param
- convert all notebooks that have `//default_exp <name>` in the first cell
    - `<name>` will be used as the js script name
- write `.js` and `.module.js`  files to a `src` directory
    - this can be overriden via the `target` param
- `.js` files will contain all code cells that start with `/**` and do not contain `import`
- `.module.js` files will contain all code cells that start with `/**` or `export`
    
Just as an example, you could convert 00_util.ipynb and save the js script to the current working directory with;

```
!python nbdev_js.py ./00_util.ipynb .
```

In [1]:
from nbdev_quick import *
nbdev_quick.init('nbdev_js')

In [2]:
%%nbdev_export
from pathlib import Path
import glob, re, nbconvert, nbformat
# TODO: describe conda env / dependencies
# TODO: add this to Dockerfile and use py notebook and command line from there

In [3]:
%%nbdev_export
def read_nb(fname):
    "Read the notebook in `fname`."
    with open(Path(fname),'r', encoding='utf8') as f: return nbformat.reads(f.read(), as_version=4)

In [4]:
%%nbdev_export
def check_re(cell, pat, code_only=True):
    "Check if `cell` contains a line with regex `pat`"
    if code_only and cell['cell_type'] != 'code': return
    if isinstance(pat, str): pat = re.compile(pat, re.IGNORECASE | re.MULTILINE)
    return pat.search(cell['source'])

In [5]:
%%nbdev_export
def _find_default_exp(nb):
    m=re.search(r'^//default_exp (\w+)', nb['cells'][0]['source'])
    if m is None: return None
    return m.group(1)

In [6]:
%%nbdev_export
def _extract_and_process_source(cell,target):
    source=cell['source']
    if check_re(cell, rf'import.*from.*(/{target}/)') is not None:
        source=source.replace(f'{target}/','')
    return source

In [7]:
cell=dict(source="import {argmax} from './src/util';",cell_type='code')
assert "import {argmax} from './util';"==_extract_and_process_source(cell,'src')
cell=dict(source=" {argmax} from './src/util';",cell_type='code')
assert " {argmax} from './src/util';"==_extract_and_process_source(cell,'src')

In [8]:
%%nbdev_export
def _notebook2script(fname, target):
    fname,target,nb = Path(fname),Path(target),read_nb(fname)
    default_exp=_find_default_exp(nb)
    if default_exp is None: return
    target.mkdir(parents=True,exist_ok=True)
    def _convert(include_regex,exclude_regex,target_file):
        print('Converting',fname,'to',target_file)
        with open(target_file, 'w') as f:
            for cell in nb['cells']:
                if check_re(cell, include_regex) is None: continue
                if exclude_regex and check_re(cell, exclude_regex) is not None: continue
                f.write(_extract_and_process_source(cell,target))
                f.write('\n\n')
    _convert(r'(^/\*\*)','(^import )',target/f'{default_exp}.js')
    _convert(r'(^/\*\*)|(^export )',None,target/f'{default_exp}.module.js')

In [9]:
%%nbdev_export
def notebook2script(fname=None, target='src'):
    for f in glob.glob('./[!_]*.ipynb' if fname is None else fname):
        _notebook2script(f, target)

Quick test to convert notebooks programatically

In [10]:
notebook2script()

Converting 00_testutil.ipynb to src/testutil.js
Converting 00_testutil.ipynb to src/testutil.module.js
Converting 10_util.ipynb to src/util.js
Converting 10_util.ipynb to src/util.module.js
Converting 20_data.ipynb to src/data.js
Converting 20_data.ipynb to src/data.module.js
Converting 30_nn.ipynb to src/nn.js
Converting 30_nn.ipynb to src/nn.module.js


In [11]:
notebook2script('./10_*.ipynb')

Converting 10_util.ipynb to src/util.js
Converting 10_util.ipynb to src/util.module.js


## Create README.md

In [12]:
%%nbdev_export
def notebook2md(fname='index.ipynb'):
    "Convert a notebook to README.md in the current working directory"
    print('Converting',fname,'to README.md')
    converter=nbconvert.MarkdownExporter()
    md,resources=converter.from_filename(fname)
    with open('README.md','w') as f: f.write(md)

## Make `nbdev_js.py` runnable from the command line

In [13]:
%%nbdev_export
try: IN_NOTEBOOK = 'google.colab' in str(get_ipython()) or 'ZMQInteractiveShell' in str(get_ipython())
except: IN_NOTEBOOK = False

In [14]:
IN_NOTEBOOK

True

In [15]:
%%nbdev_export
import argparse
if __name__ == '__main__' and not IN_NOTEBOOK:
    parser = argparse.ArgumentParser(description='Convert notebooks to js scripts.')
    parser.add_argument('fname', default=None, nargs='?', help='Pathname to glob.')
    parser.add_argument('target', default='src', nargs='?',
                        help='Name of directory to write js scripts to.')
    args = parser.parse_args()
    notebook2script(args.fname, args.target)
    if Path('index.ipynb').is_file(): notebook2md()

In [16]:
!python3 nbdev_js.py -h

usage: nbdev_js.py [-h] [fname] [target]

Convert notebooks to js scripts.

positional arguments:
  fname       Pathname to glob.
  target      Name of directory to write js scripts to.

optional arguments:
  -h, --help  show this help message and exit


In [17]:
!python3 nbdev_js.py ./00_*.ipynb

Converting 00_testutil.ipynb to src/testutil.js
Converting 00_testutil.ipynb to src/testutil.module.js
Converting index.ipynb to README.md
